In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/archive (1).zip'

# Directory to extract the dataset
extracted_path = '/content/dataset'

In [ ]:
os.makedirs(extracted_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

print("Dataset extracted.")

Dataset extracted.


In [ ]:
extracted_files = os.listdir(extracted_path)
print("Extracted files:", extracted_files)

Extracted files: ['Alzheimer_s Dataset']


In [ ]:
train_folder = '/content/dataset/Alzheimer_s Dataset/train'
test_folder = '/content/dataset/Alzheimer_s Dataset/test'


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

def copy_files(file_list, src_folder, dest_folder):
    os.makedirs(dest_folder, exist_ok=True)
    for file_name in file_list:
        src_path = os.path.join(src_folder, file_name)
        dest_path = os.path.join(dest_folder, file_name)
        shutil.copy(src_path, dest_path)

train_dataset = '/content/train_dataset'
test_dataset = '/content/test_dataset'
os.makedirs(train_dataset, exist_ok=True)
os.makedirs(test_dataset, exist_ok=True)


for class_folder in os.listdir(train_folder):
    class_path = os.path.join(train_folder, class_folder)
    if os.path.isdir(class_path):
        images = os.listdir(class_path)
        train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)  # Adjust test_size as needed

        copy_files(train_images, class_path, os.path.join(train_dataset, class_folder))
        copy_files(test_images, class_path, os.path.join(test_dataset, class_folder))

print("Dataset separation completed.")


Dataset separation completed.


In [ ]:
len(train_images)

41

In [ ]:
len(test_images)

11

In [ ]:

train_dataset = '/content/train_dataset'
test_dataset = '/content/test_dataset'

batch_size = 32
input_shape = (224, 224)
num_classes = 4
epochs = 10


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dataset,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dataset,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode='categorical'
)


Found 5117 images belonging to 4 classes.
Found 1283 images belonging to 4 classes.


In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_generator, epochs=epochs, validation_data=test_generator)


16705208/16705208 [==============================] - 0s 0us/step
Epoch 1/10
160/160 [==============================] - 668s 4s/step - loss: 1.0751 - accuracy: 0.4800 - val_loss: 1.1201 - val_accuracy: 0.3500
Epoch 2/10
160/160 [==============================] - 662s 4s/step - loss: 1.0562 - accuracy: 0.4847 - val_loss: 1.0514 - val_accuracy: 0.4988
Epoch 3/10
160/160 [==============================] - 622s 4s/step - loss: 1.0520 - accuracy: 0.4892 - val_loss: 1.0443 - val_accuracy: 0.4988
Epoch 4/10
160/160 [==============================] - 648s 4s/step - loss: 1.0441 - accuracy: 0.4901 - val_loss: 1.0578 - val_accuracy: 0.4988
Epoch 5/10
160/160 [==============================] - 609s 4s/step - loss: 1.0390 - accuracy: 0.5003 - val_loss: 1.0449 - val_accuracy: 0.4988
Epoch 6/10
160/160 [==============================] - 653s 4s/step - loss: 1.0389 - accuracy: 0.5003 - val_loss: 1.0403 - val_accuracy: 0.4988
Epoch 7/10
160/160 [==============================] - 665s 4s/step - loss: 1.

In [ ]:
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print("\nTest accuracy:", test_acc)

41/41 - 121s - loss: 1.0582 - accuracy: 0.4988 - 121s/epoch - 3s/step

Test accuracy: 0.4988308548927307


In [ ]:
from PIL import Image

In [ ]:
from keras.preprocessing import image
img_path = '/content/kk.jpg'
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

predictions = model.predict(img_array)


1/1 [==============================] - 2s 2s/step


In [ ]:
class_labels = train_generator.class_indices

predicted_class = np.argmax(predictions)

predicted_label = [k for k, v in class_labels.items() if v == predicted_class][0]

print("Predicted class:", predicted_label)


Predicted class: NonDemented
